In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
ink_map = {'C': 'Cyan', 'M': 'Magenta', 'Y': 'Yellow', 'K': 'Black'}

In [ ]:
cal_file = r"CMYKramps_p2_base.cal"
print_params = {"LightCyanValue": 0.35, "LightCyanTrans": 0.75,
                "LightMagentaValue": 0.33, "LightMagentaTrans": 0.75,
                "BlackGamma": 1.0, "CyanGamma": 1.0, "MagentaGamma": 1.0, "YellowGamma": 1.0, "CompositeGamma": 1.0}

In [ ]:
cal_file = r"CMYKramps_p2_post_LightVal.cal"
print_params = {"LightCyanValue": 0.21, "LightCyanTrans": 0.75,
                "LightMagentaValue": 0.315, "LightMagentaTrans": 0.75,
                "BlackGamma": 1.0, "CyanGamma": 1.0, "MagentaGamma": 1.0, "YellowGamma": 1.0, "CompositeGamma": 1.0}

In [ ]:
cal_file = r"LightVal_gamma_r2_p2.cal"
print_params = {"LightCyanValue": 0.21, "LightCyanTrans": 0.75,
                "LightMagentaValue": 0.315, "LightMagentaTrans": 0.75,
                "BlackGamma": 1.055, "CyanGamma": .96, "MagentaGamma": .895, "YellowGamma": .915, "CompositeGamma": 1.0}

In [ ]:
def is_float(element: any) -> bool:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False

In [ ]:
def parse_cal_blocks(path):
    blocks = {}
    current = None
    data = []
    data_format = []
    current_block = None

    with open(path) as f:
        for line in f:
            line = line.strip()

            if line.startswith("DESCRIPTOR"):
                current = line.split('"')[1]
                blocks[current] = {}
                data = []

            elif line == "BEGIN_DATA_FORMAT":
                data_format = []
                current_block = "DATA_FORMAT"

            elif line == "END_DATA_FORMAT":
                blocks[current]['data_format'] = data_format
                current_block = None
            
            elif line == "BEGIN_DATA":
                data = []
                current_block = "DATA"

            elif line == "END_DATA":
                blocks[current]['rawdata'] = np.array(data)
                current_block = None

            elif current and line and not line.startswith("#"):
                try:
                    if current_block == "DATA_FORMAT":
                        data_format.append([x for x in line.split()])
                    elif current_block == "DATA":
                        data.append([float(x) if is_float(x) else x for x in line.split()])
                except ValueError:
                    pass

    for iBlock, block in enumerate(blocks.keys()):
        blocks[block]['data'] = pd.DataFrame(blocks[block]['rawdata'])
        blocks[block]['data'].columns = blocks[block]['data_format']
    return blocks

## Load .cal

In [ ]:
cal_blocks = parse_cal_blocks(cal_file)
print("Found blocks:")
for block in cal_blocks.keys():
    print(f"\t{block}")

In [ ]:
cal_curves = cal_blocks["Argyll Device Calibration Curves"]['data']
out_curves = cal_blocks["Argyll Output Calibration Expected DE Response"]['data']

In [ ]:
in_curves = {}
in_curves['CMYK_I'] = out_curves["CMYK_I_DE"].values

for ch in ['C', 'M', 'Y', 'K']:
    I_out = out_curves["CMYK_I_DE"].values
    O_out = out_curves[f"CMYK_{ch}_DE"].values
    I_cal = cal_curves["CMYK_I"].values
    O_cal = cal_curves[f"CMYK_{ch}"].values

    #cal_corr = O_cal / I_cal
    #cal_corr = np.nan_to_num(cal_corr, nan=1.0)
    #in_curves[f"CMYK_{ch}"] = np.atleast_2d(O_out.flatten() / np.interp(I_out.flatten(), I_cal.flatten(), cal_corr.flatten())).transpose()

    in_curves[f"CMYK_{ch}"] = np.nan_to_num(np.atleast_2d(O_out.flatten() * np.interp(I_out.flatten(), O_cal.flatten(), I_cal.flatten()) / I_cal.flatten()).transpose(), nan=0.0)
    

In [ ]:
plt.figure()
for ch in ['C', 'M', 'Y', 'K']:
    plt.plot(in_curves['CMYK_I'], in_curves[f'CMYK_{ch}'], f"{ch.lower()}", label=f"{ch}")
    plt.plot([in_curves['CMYK_I'][0], in_curves['CMYK_I'][-1]], [in_curves[f'CMYK_{ch}'][0], in_curves[f'CMYK_{ch}'][-1]], f"{ch.lower()}--", label=f"{ch}")
plt.xlabel("Input I")
plt.ylabel("Output DE")
plt.title("Measured curves before calibration")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure()
for ch in ['C', 'M', 'Y', 'K']:
    plt.plot(cal_curves['CMYK_I'], cal_curves[f'CMYK_{ch}'], f"{ch.lower()}", label=f"{ch}")
plt.xlabel("Input I")
plt.ylabel("Output I")
plt.title("Calibration curves")
plt.grid()
plt.legend()
plt.show()

In [ ]:
def gamma_fit_old(I, O):
    """
    Fits O = I^g in log domain.
    """
    mask = (I > 1e-4) & (O > 1e-4)
    x = np.log(I[mask])
    y = np.log(O[mask])
    g = np.polyfit(x, y, 1)[0]
    return g


def gamma_fit(I, O, eps=1e-6):
    """
    Fit O = I^gamma using log-log least squares.
    I, O in [0, 1], with 0 = white, 1 = full ink.
    """
    mask = (I > eps) & (O > eps) & (I < 1 - eps) & (O < 1 - eps)

    logI = np.log(I[mask])
    logO = np.log(O[mask])

    # Least squares fit: logO = gamma * logI
    gamma, _, _, _ = np.linalg.lstsq(
        logI[:, None], logO, rcond=None
    )

    return float(gamma[0])


def linear_slope(I, O):
    """
    Fits O = s*I + b, returns slope s.
    """
    s, _ = np.polyfit(I, O, 1)
    return s

def linear_slope_through_point(I, O, x0, y0):
    def fit_func(x, a):
        # Curve fitting function
        return a * (x - x0) + y0  # b=0 is implied

    return curve_fit(fit_func, I, O)[0][0]


def analyze_light_channel(I, O, light_value, light_trans):
    """
    Analyze a channel with light ink (C/M).
    """

    safe_bounds = 0.01

    transition_lower = light_value * (1 - light_trans)
    transition_upper = 1.0 * light_trans
    
    dark_mask = (I > transition_upper + safe_bounds) & (I < (1 - safe_bounds))
    light_mask = (I > safe_bounds) & (I < transition_lower - safe_bounds)

    Id, Od = I[dark_mask], O[dark_mask]
    Il, Ol = I[light_mask], O[light_mask]

    g_dark = gamma_fit(Id, Od)
    s_dark = linear_slope_through_point(Id, Od, 1.0, 1.0)
    s_light = linear_slope_through_point(Il, Ol, 0.0, 0.0)

    #light_value_est = s_light / s_dark
    light_value_est = s_light * light_value

    return {
        "gamma_dark": g_dark,
        "s_dark": s_dark,
        "s_light": s_light,
        "light_value_est": light_value_est,
        "Id": Id,
        "Od": Od,
        "Il": Il,
        "Ol": Ol,
    }


def analyze_dark_channel(I, O):
    """
    Analyze Y or K (no light ink).
    """
    mask = (I > 0.05) & (I < 0.95)
    g = gamma_fit(I[mask], O[mask])
    return {"gamma": g}

In [ ]:
def plot_channel(name, I, O, result):
    plt.figure(figsize=(6, 5))
    plt.plot(I, O, "k.", label="Measured")

    if "Id" in result:
        Id, Od = result["Id"], result["Od"]
        Il, Ol = result["Il"], result["Ol"]

        g = result["gamma_dark"]
        s_dark = result["s_dark"]
        s_light = result["s_light"]

        I_fit = np.linspace(Id.min(), Id.max(), 200)
        plt.plot(I_fit, I_fit ** g, "r-", label=f"Dark gamma fit (g={1/g:.2f})")

        I_fit = np.linspace(Il.min(), Il.max(), 200)
        plt.plot(I_fit, s_light * I_fit, "b-", label="Light slope fit")

        I_fit = np.linspace(Id.min(), Id.max(), 200)
        plt.plot(I_fit, (s_dark * (I_fit-1.0))+1.0, "g-", label="Dark slope fit")

    else:
        g = result["gamma"]
        I_fit = np.linspace(0.01, 1.0, 200)
        plt.plot(I_fit, I_fit ** g, "r-", label=f"Gamma fit (g={1/g:.2f})")

    plt.xlabel("Nominal input")
    plt.ylabel("Argyll device output")
    plt.title(name)
    plt.legend()
    plt.grid(True)

    #out = Path(outdir) / f"{name}.png"
    #plt.savefig(out, dpi=150)
    #plt.close()

In [ ]:
results_trans = {}

# Light ink channels
for ch, light in [("C", "LightCyan"), ("M", "LightMagenta")]:
    I = in_curves["CMYK_I"]
    O = in_curves[f"CMYK_{ch}"] / in_curves[f"CMYK_{ch}"][-1]

    res = analyze_light_channel(
        I,
        O,
        print_params[f"{light}Value"],
        print_params[f"{light}Trans"],
    )

    results_trans[ch] = res
    plot_channel(ch, I, O, res)

# Y and K
for ch in ["Y", "K"]:
    I = in_curves["CMYK_I"]
    O = in_curves[f"CMYK_{ch}"] / in_curves[f"CMYK_{ch}"][-1]
    res = analyze_dark_channel(I, O)
    results_trans[ch] = res
    plot_channel(ch, I, O, res)

In [ ]:
results_light = {}
results_all = {}

# Light ink channels
for ch, light in [("C", "LightCyan"), ("M", "LightMagenta")]:
    I = cal_blocks["Argyll Device Calibration Curves"]['data']["CMYK_I"].values
    O = cal_blocks["Argyll Device Calibration Curves"]['data'][f"CMYK_{ch}"].values

    res = analyze_light_channel(
        I,
        O,
        print_params[f"{light}Value"],
        print_params[f"{light}Trans"],
    )

    results_light[ch] = res
    plot_channel(ch, I, O, res)

# Y and K
for ch in ["C", "M", "Y", "K"]:
    I = cal_blocks["Argyll Device Calibration Curves"]['data']["CMYK_I"].values
    O = cal_blocks["Argyll Device Calibration Curves"]['data'][f"CMYK_{ch}"].values
    res = analyze_dark_channel(I, O)
    results_all[ch] = res
    plot_channel(ch, I, O, res)

In [ ]:
for ch in ["C", "M"]:
    print(f"{ch}:")
    print(f"  Dark gamma multiplier ≈ {(print_params[ink_map[ch] + 'Gamma'] / results_light[ch]['gamma_dark']):.3f}")
    print(f"  Overall Gamma multiplier ≈ {(print_params[ink_map[ch] + 'Gamma'] / results_all[ch]['gamma']):.3f}")
    print(f"  LightValue estimate   ≈ {results_trans[ch]['light_value_est']:.3f}")
    print()

for ch in ["Y", "K"]:
    print(f"{ch}:")
    print(f"  Overall Gamma multiplier ≈ {(print_params[ink_map[ch] + 'Gamma'] / results_all[ch]['gamma']):.3f}")
    print()